In [3]:
%matplotlib inline

import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sparse
from scipy.stats import pearsonr

sys.path.append('../../Code/')
from utils import read_dataset_log


from matplotlib.patches import Patch
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from matplotlib.patches import Rectangle
from matplotlib.pyplot import rc_context

In [4]:
metagepinfo = read_dataset_log(tabname='Current Meta-program Paths').loc[0,:]
metagepinfo

clustering_fn                      /data/srlab1/TCAT/Data/TCAT/cluster_groups.pai...
clustering_with_singletons_fn      /data/srlab1/TCAT/Data/TCAT/cluster_groups.pai...
merged_hvgs                        /data/srlab1/TCAT/Data/TCAT/MergedHVG.UnionAll...
tpm_norm_spectra                   /data/srlab1/TCAT/Data/TCAT/merged_spectra.Gen...
tpm_norm_spectra_withsingletons    /data/srlab1/TCAT/Data/TCAT/merged_spectra.Gen...
scores_spectra                     /data/srlab1/TCAT/Data/TCAT/merged_spectra.Gen...
Name: 0, dtype: object

In [8]:
clus_df_labeled_clean = pd.read_csv(metagepinfo.at['clustering_with_singletons_fn'], sep='\t', index_col=0)

In [12]:
cancer_singletons = clus_df_labeled_clean.drop('Pan-Cancer', axis=1).isnull().sum(axis=1)==6

In [13]:
clus_df_labeled_clean.loc[cancer_singletons, :]

,AMP-RA,UK-Covid,Pan-Tissue,HIV-Vaccine,COMBAT,TBRU,Pan-Cancer
Pan-Cancer:CD4/CD20_Single,NaN,NaN,NaN,NaN,NaN,NaN,Pan-Cancer:CD4/CD20
Pan-Cancer:Tex_Single,NaN,NaN,NaN,NaN,NaN,NaN,Pan-Cancer:Tex
Pan-Cancer:CTSH/FURIN_Single,NaN,NaN,NaN,NaN,NaN,NaN,Pan-Cancer:CTSH/FURIN
Pan-Cancer:Treg/TNFRSF18_Single,NaN,NaN,NaN,NaN,NaN,NaN,Pan-Cancer:Treg/TNFRSF18
Pan-Cancer:Doublet_epithelial_Single,NaN,NaN,NaN,NaN,NaN,NaN,Pan-Cancer:Doublet_epithelial
Pan-Cancer:Doublet_Fibroblast_Single,NaN,NaN,NaN,NaN,NaN,NaN,Pan-Cancer:Doublet_Fibroblast
Pan-Cancer:RXRG/WLS_Single,NaN,NaN,NaN,NaN,NaN,NaN,Pan-Cancer:RXRG/WLS
Pan-Cancer:FXYD6/CAV1_Single,NaN,NaN,NaN,NaN,NaN,NaN,Pan-Cancer:FXYD6/CAV1
Pan-Cancer:IGFBP5/FN1_Single,NaN,NaN,NaN,NaN,NaN,NaN,Pan-Cancer:IGFBP5/FN1
Pan-Cancer:CellCycle-Histone1_Single,NaN,NaN,NaN,NaN,NaN,NaN,Pan-Cancer:CellCycle-Histone1


In [26]:
pancancer_singleton_names = clus_df_labeled_clean.loc[cancer_singletons, 'Pan-Cancer']
pancancer_singleton_names = [x.split(':')[1] for x in pancancer_singleton_names]
pancancer_singleton_names

['CD4/CD20',
 'Tex',
 'CTSH/FURIN',
 'Treg/TNFRSF18',
 'Doublet_epithelial',
 'Doublet_Fibroblast',
 'RXRG/WLS',
 'FXYD6/CAV1',
 'IGFBP5/FN1',
 'CellCycle-Histone1',
 'Metabolic',
 'CCL17/CCL19',
 'S100A2/KRT6A',
 'PIP/PI15']

In [18]:
params = read_dataset_log(tabname='Current Dataset Paths')
params.index = params['dataset']
params

,dataset,usage_fn,gene_scores_fn,gene_tpm_fn,tcat_fn,manual_gating_fn,metadata_fn,Processing notebook path,cNMF notebook path,cluster_col,name,cnmf_dir,k,dt,processed_forcnmf_fn,raw_counts_fn
dataset,,,,,,,,,,,,,,,,
AMP-RA,AMP-RA,/data/srlab1/mcurtis/GSK/tcell_proliferation/A...,/data/srlab1/mcurtis/GSK/tcell_proliferation/A...,/data/srlab1/mcurtis/GSK/tcell_proliferation/A...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage_2023070...,NaN,/data/srlab1/mcurtis/GSK/tcell_proliferation/A...,/data/srlab1/mcurtis/GSK/tcell_proliferation/n...,/data/srlab1/mcurtis/GSK/tcell_proliferation/n...,cluster_name,amp_HarmonyRNA,/data/srlab1/mcurtis/GSK/tcell_proliferation/A...,35.0,0_2,/data/srlab1/mcurtis/GSK/tcell_proliferation/A...,/data/srlab1/mcurtis/GSK/tcell_proliferation/A...
Pan-Cancer,Pan-Cancer,/data/srlab1/mcurtis/GSK/tcell_proliferation/z...,/data/srlab1/mcurtis/GSK/tcell_proliferation/z...,/data/srlab1/mcurtis/GSK/tcell_proliferation/z...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage_2023070...,NaN,/data/srlab1/mcurtis/GSK/tcell_proliferation/z...,/data/srlab1/mcurtis/GSK/tcell_proliferation/n...,/data/srlab1/mcurtis/GSK/tcell_proliferation/n...,meta.cluster,PanCancer_HarmonyRNA.20kgenes10X,/data/srlab1/mcurtis/GSK/tcell_proliferation/z...,38.0,0_15,/data/srlab1/mcurtis/GSK/tcell_proliferation/z...,/data/srlab1/mcurtis/GSK/tcell_proliferation/z...
TBRU,TBRU,/data/srlab1/mcurtis/GSK/tcell_proliferation/T...,/data/srlab1/mcurtis/GSK/tcell_proliferation/T...,/data/srlab1/mcurtis/GSK/tcell_proliferation/T...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage_2023070...,NaN,/data/srlab1/mcurtis/GSK/tcell_proliferation/T...,/data/srlab1/mcurtis/GSK/tcell_proliferation/n...,/data/srlab1/mcurtis/GSK/tcell_proliferation/n...,cluster_name,TBRU_HarmonyRNA,/data/srlab1/mcurtis/GSK/tcell_proliferation/T...,35.0,0_2,/data/srlab1/mcurtis/GSK/tcell_proliferation/T...,/data/srlab1/mcurtis/GSK/tcell_proliferation/T...
HIV-Vaccine,HIV-Vaccine,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage_2023070...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/Manu...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Analy...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Analy...,celltype.l2,T_learnHarmonyRNA_RefitBoth_20221022,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,35.0,0_2,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...
UK-Covid,UK-Covid,/data/srlab1/mcurtis/GSK/tcell_proliferation/C...,/data/srlab1/mcurtis/GSK/tcell_proliferation/C...,/data/srlab1/mcurtis/GSK/tcell_proliferation/C...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage_2023070...,NaN,/data/srlab1/dk718/COVID_PBMC/Data/Haniffa/han...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Analy...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Analy...,full_clustering,Haniffa_T_Script_learnHarmonyRNA_RefitBoth,/data/srlab1/mcurtis/GSK/tcell_proliferation/C...,40.0,0_2,/data/srlab1/dk718/COVID_PBMC/Data/Haniffa/han...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/hani...
COMBAT,COMBAT,/data/srlab1/mcurtis/GSK/tcell_proliferation/C...,/data/srlab1/mcurtis/GSK/tcell_proliferation/C...,/data/srlab1/mcurtis/GSK/tcell_proliferation/C...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage_2023070...,/data/srlab1/TCAT/Data/PerDataset/COMBAT/Manua...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Analy...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Analy...,Annotation_minor_subset,T_Script_learnHarmonyRNA_RefitBoth,/data/srlab1/mcurtis/GSK/tcell_proliferation/C...,44.0,0_2,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/TCAT/Data/PerDataset/COMBAT/COMBA...
Pan-Tissue,Pan-Tissue,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/dk718/Tcell_cNMF_Comparison/Data/...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage_2023070...,NaN,/data/srlab1/dk

In [22]:
pancancerusage = pd.read_csv(params.at['Pan-Cancer', 'usage_fn'], sep='\t', index_col=0)

In [34]:
pancancerscores = pd.read_csv(params.at['Pan-Cancer', 'gene_scores_fn'], sep='\t', index_col=0).T

In [29]:
pancancerusage.sum(axis=0).sort_values(ascending=False)

Translation/T_Naive    22838.991141
YPEL5/CREM             19789.538003
Memory                 18580.671303
CD8_class2_HLA         18329.625422
Cytoskeleton           13919.760649
Cytotoxic              12848.521096
KLF6/CXCR4             12068.952122
CD4/CD20               10508.619708
NME1/FABP5              9918.930492
Treg                    9805.438740
IEG                     9619.306425
Tex                     7595.902598
MAIT                    7298.024050
HeatShock               6017.981631
CTSH/FURIN              5879.161723
Treg/TNFRSF18           5501.844040
HLA                     3770.575847
EBI3                    3665.009685
Tfh                     3191.415629
ISG                     3092.154987
gdT/Tk                  3021.373949
CellCycle-Sphase        2555.526755
RXRG/WLS                2538.804073
CellCycle-Histone2      2056.420192
Doublet_epithelial      1733.544243
Doublet_Fibroblast      1644.992665
CellCycle-G2M           1356.482915
FXYD6/CAV1              1320

In [30]:
pancancerusage.sum(axis=0).sort_values(ascending=False)[pancancer_singleton_names]

CD4/CD20              10508.619708
Tex                    7595.902598
CTSH/FURIN             5879.161723
Treg/TNFRSF18          5501.844040
Doublet_epithelial     1733.544243
Doublet_Fibroblast     1644.992665
RXRG/WLS               2538.804073
FXYD6/CAV1             1320.835232
IGFBP5/FN1              639.939024
CellCycle-Histone1     1193.670426
Metabolic               591.112552
CCL17/CCL19             422.923841
S100A2/KRT6A            262.275021
PIP/PI15                125.399485
dtype: float64

In [36]:
pancancerscores['Metabolic'].sort_values(ascending=False).head(20)

GPX3        0.037908
PCK1        0.035730
MIOX        0.035155
GATM        0.032081
PDZK1IP1    0.030542
BBOX1       0.026705
HMGCS2      0.026250
ASS1        0.025130
DPEP1       0.024896
UGT2B7      0.024588
BHMT        0.023606
CXCL14      0.021457
RBP5        0.021416
HPD         0.019890
FOLR1       0.019210
APOE        0.018681
PDZK1       0.018069
PRAP1       0.017124
CYP17A1     0.016865
SLC5A12     0.016776
Name: Metabolic, dtype: float64

In [3]:
clus_fn = tcat_dir + 'cluster_groups.pairwiseCorr.Z_HVGunion.20230627.txt'
clus_df_labeled_clean = pd.read_csv(clus_fn, sep ='\t', index_col=0)
clus_df_labeled_clean.head()

,AMP-RA,UK-Covid,Pan-Tissue,HIV-Vaccine,COMBAT,TBRU,Pan-Cancer
CellCycle-G2M_CellCycle_G2M,AMP-RA:CellCycle-G2M,UK-Covid:CellCycle_G2M,Pan-Tissue:CellCycle-G2M,HIV-Vaccine:CellCycle_G2M,COMBAT:CellCycle_G2M,TBRU:CellCycle_G2M,Pan-Cancer:CellCycle-G2M
ISG,AMP-RA:ISG,UK-Covid:ISG,Pan-Tissue:ISG,HIV-Vaccine:ISG,COMBAT:ISG,TBRU:ISG,Pan-Cancer:ISG
gdT,NaN,UK-Covid:gdT,NaN,HIV-Vaccine:gdT,COMBAT:gdT,TBRU:gdT,NaN
Metallothionein,AMP-RA:Metallothionein,UK-Covid:Metallothionein,Pan-Tissue:Metallothionein,NaN,COMBAT:Metallothionein,NaN,Pan-Cancer:Metallothionein
CellCycle-Sphase_CellCycle_S_DTL/FAM111B,AMP-RA:CellCycle-Sphase,UK-Covid:CellCycle_S,Pan-Tissue:CellCycle-Sphase,HIV-Vaccine:CellCycle_S,COMBAT:CellCycle_S,TBRU:DTL/FAM111B,Pan-Cancer:CellCycle-Sphase


In [4]:
clus_df_labeled_clean.loc['EBI3', :]

AMP-RA             AMP-RA:EBI3
UK-Covid                   NaN
Pan-Tissue                 NaN
HIV-Vaccine                NaN
COMBAT                     NaN
TBRU                       NaN
Pan-Cancer     Pan-Cancer:EBI3
Name: EBI3, dtype: object

In [4]:
all_singletons = clus_df_labeled_clean.index[(~clus_df_labeled_clean.isnull()).sum(axis=1)==1]
dataset = clus_df_labeled_clean.loc[all_singletons, :].apply(lambda x: x[x.notnull()].index[0], axis=1)

In [5]:
dataset.index[dataset=='AMP-RA']

Index(['AMP-RA:Tph_Single', 'AMP-RA:Doublet_Fibroblast_Single',
       'AMP-RA:SERPINE2_Single', 'AMP-RA:Doublet_Plasma_Single',
       'AMP-RA:Hypoxia_Single', 'AMP-RA:Doublet_pDC_Single',
       'AMP-RA:Doublet_Mast_Single', 'AMP-RA:Doublet_cardiomyocyte_Single'],
      dtype='object')

In [28]:
single_prgs_keep = ['AMP-RA:Tph',
 #'AMP-RA:SERPINE2', ## maybe, maybe arbitrary Tfh split
 
 'Pan-Cancer:Tex',
 'Pan-Cancer:CTSH/FURIN',
 'Pan-Cancer:Treg/TNFRSF18', ### maybe, defines a subset of Treg clusters
 
 'TBRU:ICOS/CCR4', ### maybe, defines clusters
 
 'HIV-Vaccine:?KLRC2/GNLY',
 'HIV-Vaccine:?NK-Like',
 'HIV-Vaccine:Doublet_Plasmablast',

 'Pan-Tissue:?METRNL/CREM/LDLRAD4', 
 'Pan-Tissue:CD244+CD8',
 'Pan-Tissue:NK-like',
 'Pan-Tissue:?CH25H/DDIT4/SOCS3', ### maybe, probably noise based on correlated IEG prgs, but high usage
 #'Pan-Tissue:?PlasmaDoublet/WFDC2/SCGB3A1' ### maybe, high usage but likely naive/translation split
 
]

In [36]:
[x for x in all_singletons if]

['AMP-RA:Tph_Single',
 'AMP-RA:Doublet_Fibroblast_Single',
 'AMP-RA:SERPINE2_Single',
 'AMP-RA:Doublet_Plasma_Single',
 'AMP-RA:Hypoxia_Single',
 'AMP-RA:Doublet_pDC_Single',
 'AMP-RA:Doublet_Mast_Single',
 'AMP-RA:Doublet_cardiomyocyte_Single',
 'Pan-Cancer:CD4/CD20_Single',
 'Pan-Cancer:Tex_Single',
 'Pan-Cancer:CTSH/FURIN_Single',
 'Pan-Cancer:Treg/TNFRSF18_Single',
 'Pan-Cancer:Doublet_epithelial_Single',
 'Pan-Cancer:Doublet_Fibroblast_Single',
 'Pan-Cancer:RXRG/WLS_Single',
 'Pan-Cancer:FXYD6/CAV1_Single',
 'Pan-Cancer:IGFBP5/FN1_Single',
 'Pan-Cancer:CellCycle-Histone1_Single',
 'Pan-Cancer:Metabolic_Single',
 'Pan-Cancer:CCL17/CCL19_Single',
 'Pan-Cancer:S100A2/KRT6A_Single',
 'Pan-Cancer:PIP/PI15_Single',
 'TBRU:T_Naive_Single',
 'TBRU:ICOS/CCR4_Single',
 'TBRU:PoorQuality2_Single',
 'TBRU:ICA1/TIGIT_Single',
 'TBRU:SCN2A/BARX1_Single',
 'HIV-Vaccine:?MTRNR2L12_Single',
 'HIV-Vaccine:?TXNIP/H3F3B_Single',
 'HIV-Vaccine:?KLRC2/GNLY_Single',
 'HIV-Vaccine:?NK-Like_Single',
 'HIV

In [29]:
ampra_scores = pd.read_csv(params.loc['AMP-RA', 'gene_scores_fn'], sep='\t', index_col=0).T
ampra_scores.columns

Index(['Ribo', 'PoorQuality', 'CD8_class2_HLA', 'KLF6/CXCR4', 'Ribo/T_Naive',
       'YPEL5/CREM', 'IEG', 'Tfh', 'Cytoskeleton', 'Cytotoxic', 'CD4_Memory',
       'Treg', 'gdT', 'Tph', 'Doublet_Fibroblast', 'NME1/FABP5', 'MAIT',
       'HeatShock', 'ISG', 'SERPINE2', 'EBI3', 'CellCycle-Sphase',
       'Doublet_DC', 'IL17A/IFNG', 'CD38', 'CellCycle-Histone',
       'Doublet_Plasma', 'Hypoxia', 'Doublet_Bcell', 'CellCycle-G2M',
       'Metallothionein', 'Doublet_pDC', 'Doublet_Mast',
       'Doublet_cardiomyocyte', 'Doublet_RBC'],
      dtype='object')

In [30]:
ampra_scores['Doublet_RBC'].sort_values(ascending=False).head(25)

HBA1           0.108127
HBB            0.091915
HBA2           0.089496
ALAS2          0.074850
HBQ1           0.042421
AHSP           0.025794
HBD            0.024121
MYL4           0.019584
SNCA           0.006702
HEYL           0.003738
CCNJL          0.003673
SLC25A37       0.002500
MTSS1L         0.002396
TSSK4          0.002375
HOXA5          0.002024
BMP2           0.002018
DIRAS3         0.001884
SLC4A1         0.001765
GLRA1          0.001654
FPGT-TNNI3K    0.001610
CTAG2          0.001566
DRD4           0.001509
PDE6G          0.001460
B3GNT7         0.001458
YBX3           0.001441
Name: Doublet_RBC, dtype: float64

In [31]:
ampra_scores['Tph'].sort_values(ascending=False).head(25)

LAG3             0.008872
LAIR2            0.006203
SLC27A2          0.005814
TNFRSF18         0.005287
CD7              0.005086
PDCD1            0.005013
CD4              0.004981
LINC01943        0.004979
SRGN             0.004917
AB_CD279/PD-1    0.004883
CTLA4            0.004838
PTMS             0.004582
HAVCR2           0.004406
ZBED2            0.004306
TIMP1            0.004298
RDH10            0.004256
SMOX             0.004130
SPOCK2           0.004117
BST2             0.004010
PLPP1            0.003914
ADGRG1           0.003850
ETV7             0.003820
MYO7A            0.003806
MYO1E            0.003728
CXCL13           0.003716
Name: Tph, dtype: float64

In [7]:
ampra_scores = pd.read_csv(params.loc['AMP-RA', 'gene_scores_fn'], sep='\t', index_col=0).T
ampra_scores.columns

Index(['Ribo', 'PoorQuality', 'CD8_class2_HLA', 'KLF6/CXCR4', 'Ribo/T_Naive',
       'YPEL5/CREM', 'IEG', 'Tfh', 'Cytoskeleton', 'Cytotoxic', 'CD4_Memory',
       'Treg', 'gdT', 'Tph', 'Doublet_Fibroblast', 'NME1/FABP5', 'MAIT',
       'HeatShock', 'ISG', 'SERPINE2', 'EBI3', 'CellCycle-Sphase',
       'Doublet_DC', 'IL17A/IFNG', 'CD38', 'CellCycle-Histone',
       'Doublet_Plasma', 'Hypoxia', 'Doublet_Bcell', 'CellCycle-G2M',
       'Metallothionein', 'Doublet_pDC', 'Doublet_Mast',
       'Doublet_cardiomyocyte', 'Doublet_RBC'],
      dtype='object')

In [13]:
pancancer_scores = pd.read_csv(params.loc['Pan-Cancer', 'gene_scores_fn'], sep='\t', index_col=0).T
pancancer_scores.columns

Index(['Translation/T_Naive', 'YPEL5/CREM', 'Memory', 'CD8_class2_HLA',
       'Cytotoxic', 'Cytoskeleton', 'KLF6/CXCR4', 'Treg', 'IEG', 'CD4/CD20',
       'NME1/FABP5', 'Tex', 'MAIT', 'HeatShock', 'CTSH/FURIN', 'Treg/TNFRSF18',
       'EBI3', 'Doublet_epithelial', 'HLA', 'Tfh', 'ISG', 'gdT/Tk',
       'Doublet_Fibroblast', 'RXRG/WLS', 'CellCycle-Sphase',
       'CellCycle-Histone2', 'CellCycle-G2M', 'FXYD6/CAV1', 'Doublet_Bcell',
       'IGFBP5/FN1', 'CellCycle-Histone1', 'Metabolic', 'Metallothionein',
       'CCL17/CCL19', 'S100A2/KRT6A', 'Doublet_RBC', 'PIP/PI15',
       'CRHBP/CLEC4G'],
      dtype='object')

In [11]:
ampra_scores.loc[:,'EBI3'].sort_values(ascending=False).head(20)

EBI3           0.014426
TNFRSF4        0.008296
TNFRSF18       0.007227
KRT7           0.006643
BHLHE40-AS1    0.006606
NINJ1          0.005957
CAVIN3         0.005881
TNFRSF8        0.005452
G0S2           0.005133
SDC4           0.005053
HMSD           0.004891
PKM            0.004678
COL9A2         0.004660
MIR155HG       0.004631
GNG8           0.004566
PTP4A3         0.004503
ZG16B          0.004398
RELB           0.004370
SORD           0.004309
HSD11B1        0.004280
Name: EBI3, dtype: float64

In [14]:
pancancer_scores.loc[:,'EBI3'].sort_values(ascending=False).head(20)

TNFRSF4     0.009718
EBI3        0.008445
CCL22       0.008171
PTP4A3      0.007520
TNFRSF18    0.006385
PKM         0.006278
BCL2L1      0.005117
TNFRSF8     0.004786
NINJ1       0.004676
TNFRSF9     0.004178
IL2RA       0.004088
NFKB2       0.003985
CCR8        0.003976
REL         0.003947
SYNGR2      0.003890
CD82        0.003853
TRAF1       0.003785
VDR         0.003748
CD83        0.003576
KCNN4       0.003563
Name: EBI3, dtype: float64